# Exploration for Op Sem

## AST

First let's flesh out what sorts of expressions we need.

TERMS
- Everything in STLC: 
    - Constants and variables
    - lambdas `fun x: A -> e`, application `e e'`, 
    - integers, integer operations (+ - * / Unary-), eliminators (=, >, <, <=, >=)
    - booleans, boolean operations, if/then/else, Operators (and, or, not)
    - let variable bindings aliasing lambdas (No polymorphic) `let x: A = e in e'`
    - Maybe later: product and sum types.
- Top level definitions as statements.
    - variables `let x: A = e in e'`
    - Maybe later: functions (maybe not yet, add later as syntactic sugar) `let f x = e in e'`
    - recursive functions `let rec f: A = e`
- Extensions from CMTT
    - box(judgement) where judgement = context |- term
    - let box unboxing
    - closures with "with" statements

Inspired from https://www.cs.cornell.edu/courses/cs3110/2014sp/hw/4/doc/Ast.html and https://github.com/Timothy-G-Griffin/cc_cl_cam_ac_uk/blob/master/slang/ast.ml

Doubts: 

- Should I separate variable from meta-variable? The advantage of separating is easier implementation; the advantage of not doing so is the ease of extending the language to multi-level later on without needing to affect the AST. 
- To what extent should I leave stuff to the type checker and to what extent should I throw parse errors in the first place? How general should I make my AST?
- Unclear as of now whether I might need to specify the type for metavariables.

In [3]:
type identifier = string

type typ = TBool | TInt | TFun of typ * typ | TBox of context * typ | TProd of typ * typ | TSum of typ * typ
and identifier_defn = identifier * typ (*x : A*)
and context = identifier_defn list (*\Psi*)

type binary_op = ADD | SUB | MUL | DIV | MOD | EQ | NEQ | GTE | GT | LTE | LT | AND | OR (*e op e'*)

type unary_op = NEG | NOT (*uop e*)

type constant = Integer of int | Boolean of bool

type expression = Identifier of identifier (*x*)
                | Constant of constant (*c*)
                | UnaryOp of unary_op * expression (*uop e*)
                | BinaryOp of binary_op * expression * expression (*e op e'*)
                | Prod of expression * expression (*(e, e')*)
                | Fst of expression (*fst e*)
                | Snd of expression (*snd e*)
                | Left of typ * typ * expression (*L[A,B] e*)
                | Right of typ * typ * expression (*R[A,B] e*)
                | Match of expression * expression * expression (*match e with L (x: A) -> e' | R (y: B) -> e'' translates to 1 expr and 2 lambdas*)
                | Lambda of identifier_defn * expression (*fun (x : A) -> e*)
                | Application of expression * expression (*e e'*)
                | IfThenElse of expression * expression * expression (*if e then e' else e''*)
                | LetBinding of identifier_defn * expression * expression (*let x: A = e in e'*)
                | LetRec of identifier_defn * expression * expression (*let rec f: A->B = e[f] in e'*)
                | Box of context * expression (*box (x:A, y:B |- e)*)
                | LetBox of identifier * expression * expression (*let box u = e in e'*)
                | Closure of identifier * (expression list) (*u with (e1, e2, e3, ...)*)

type directive = Reset | Env | Quit (*inspired from Cornell course, used as directives to the REPL loop*)

type top_level_defn = Definition of identifier_defn * expression
                    | Expression of expression
                    | Directive of directive 

type identifier = string


type typ =
    TBool
  | TInt
  | TFun of typ * typ
  | TBox of context * typ
  | TProd of typ * typ
  | TSum of typ * typ
and identifier_defn = identifier * typ
and context = identifier_defn list


type binary_op =
    ADD
  | SUB
  | MUL
  | DIV
  | MOD
  | EQ
  | NEQ
  | GTE
  | GT
  | LTE
  | LT
  | AND
  | OR


type unary_op = NEG | NOT


type constant = Integer of int | Boolean of bool


type expression =
    Identifier of identifier
  | Constant of constant
  | UnaryOp of unary_op * expression
  | BinaryOp of binary_op * expression * expression
  | Prod of expression * expression
  | Fst of expression
  | Snd of expression
  | Left of typ * typ * expression
  | Right of typ * typ * expression
  | Match of expression * expression * expression
  | Lambda of identifier_defn * expression
  | Application of expression * expression
  | IfThenElse of expression * expression * expression
  | LetBinding of identifier_defn * expression * expression
  | LetRec of identifier_defn * expression * expression
  | Box of context * expression
  | LetBox of identifier * expression * expression
  | Closure of identifier * expression list


type directive = Reset | Env | Quit


type top_level_defn =
    Definition of identifier_defn * expression
  | Expression of expression
  | Directive of directive


In [1]:
let rec pow n x = if n = 0 then 1 else x * pow (n-1) x;;

val pow : int -> int -> int = <fun>


In [2]:
pow 2 3;;

- : int = 9
